### Data

Downloaded from: https://www.kaggle.com/zynicide/wine-reviews

### Question 1.

Summary of data set: A row represents information about a bottle of wine including country where it is produced, description written by a sommelier, score points rated in a wine review site, price of a bottle and etc.

Essential columns:
* country
* description

Suppose that we want to discover the unique words used in the description of wines produced in a single country. Since the words used rarely in the descriptions are, however, usually uninformative, we are going to count those frequently occur in wine descriptions of each country.

Our strategy to analyze the wine data set is as follows: 
(1) Count how many times each word appears in the descriptions about the wine bottles produced in each country.
(2) For each country, calculate the set of top-100 frequent words shown in descriptions. 
(3) Then, find the words that appear in the "top-100 frequent word set" of a single country.

### 문제 1.

데이터 셋 요약: 각 행의 데이터는 와인에 대한 정보, 즉, 생산국, 소믈리에가 작성한 설명문, 와인 리뷰 사이트에 따른 평점, 병 당 가격 등의 정보를 담고있다.

본 문제에 사용되는 핵심 컬럼은 다음과 같다.
* country
* decription

와인 설명 글에 사용되는 단어 중에서 오직 한 나라의 와인들을 설명하기 위해서만 사용되는 독특한 단어를 찾고 싶다고 생각해보자. 그러나 나라별로 흔히 사용되지 않는 단어를 제외하고 여러번 등장하는 단어들 사이에서 그러한 독특한 단어를 찾기위해서 나라별로 빈번히 나타나는 단어들을 뽑아서 그중에 오직 한나라의 와인 설명에만 사용되는 단어를 뽑고자 한다.

아래 코드에서 데이터를 분석하는 과정은 크게 다음과 같다: (1) 각 나라별로 와인 설명문에 등장하는 각 단어들의 출현 빈도를 센다. (2) 각 나라별로 출현빈도수 에 따라 순위 100개의 단어를 뽑는다. (3) 오직 한 나라의 출현빈도수 100위안에 드는 단어를 뽑는다.

In [7]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate()

# get context from the session
sc = spark.sparkContext

(1) 우선 winemag-data-130k-v2.csv 파일을 읽어 컬럼 "country", "description" 만을 남겨 df란 이름의 데이터프레임에 저장하시오 (5점)

In [8]:
df = spark.read.csv("winemag-data-130k-v2.csv", inferSchema=True, header=True, escape='"')
df = df.select("country", "description")
print(df.count())
df.show(3)

129975
+--------+--------------------+
| country|         description|
+--------+--------------------+
|   Italy|Aromas include tr...|
|Portugal|This is ripe and ...|
|      US|Tart and snappy, ...|
+--------+--------------------+
only showing top 3 rows



(2) df에 ***description이 None이거나 country가 None인*** row가 있는지 확인하고 (즉, 그러한 row의 개수를 세어보고) 있으면 (즉, 둘중에 하나라도 None이라면) filter를 이용해 이를 제거하여 그 결과의 RDD를 rdd1이란 이름의 변수에 저장하시오 (10점)

In [9]:
df.rdd.map(lambda row: 1 if sum([c == None for c in row]) > 0 else 0  ).reduce(lambda x, y : x + y)

64

In [10]:
df = df.na.drop('any')
df.count()

129911

In [11]:
rdd1 = df.rdd
rdd1.take(3)

[Row(country='Italy', description="Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."),
 Row(country='Portugal', description="This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016."),
 Row(country='US', description='Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.')]

(3) rdd에 있는 각 Row의 description을 처리하여 나라별 단어의 출연횟수를 카운트하기 위해 적절한 flatMap을 이용한 transformation을 수행하시오. 즉, description을 띄어쓰기 (" ") 단위로 잘라서 각 단어마다 country 와 단어의 쌍(tuple)을 키로하고 값을 1로 하는 키-값 쌍을 생성하도록 flatMap을 작성하시오. 더불어, 단어는 대소문자를 구분하지 않기 위해 모두 소문자로 바꾸어 두시오. flatMap 수행 결과 RDD는 rdd2 란 이름의 변수에 저장하시오. (10점)

In [12]:
rdd2=rdd1.flatMap(lambda x: [((x[0].lower(), w.lower()), 1) for w in x[1].split(" ")])
rdd2.take(1)

[(('italy', 'aromas'), 1)]

(4) rdd2의 키 (즉, country와 단어의 쌍의 tuple)를 카운트하는 transformation을 작성하시오. 그 결과는 rdd3에 저장하시오 (10점)

In [13]:
rdd3 = rdd2.reduceByKey(lambda x,y: x + y)
rdd3.take(1)

[(('italy', 'aromas'), 11805)]

(5) groupByKey를 이용하여 나라별로 나타나는 단어와 그 count 정보를 모으고자 한다. 우선은 rdd3을 transformation하여 country를 key로 하고 단어 및 단어의 count의 튜플을 value로하는 RDD를 생성하여 rdd4에 저장하자 (10점) 

In [20]:
rdd4 = rdd3.map(lambda x: (x[0][0], ( x[0][1], x[1] ) ) ) 
rdd4.take(1)

[('italy', ('aromas', 11805))]

(6) rdd4 에 groupByKey를 적용하여 나라별로 (word, count) 튜플을 모은다. 모든 튜플의 리스트는 count의 내림차순으로 정렬하여 그 결과 RDD를 rdd5에 저장한다 (즉, rdd5는 키가 <U>country</U>, 값은 <U>count 내림차순으로 정렬된 (word, count) 튜플의 리스트</U>가 된다) (15점)

힌트: 튜플의 리스트를 second element순으로 정렬하는 방법은 아래 예제코드를 참고하면된다.

````python
sorted([('abc', 121),('abc', 231),('abc', 148), ('abc',221)], key=lambda x: x[1], reverse=True)
````

In [26]:
rdd5 = rdd4.groupByKey().map(lambda x: (x[0], sorted(list(x[1]), key=lambda x: x[1], reverse= True) ) )
rdd5.take(1)

[('italy',
  [('and', 52395),
   ('the', 32396),
   ('of', 30661),
   ('a', 21991),
   ('with', 15931),
   ('this', 14488),
   ('aromas', 11805),
   ('palate', 9330),
   ('is', 8604),
   ('wine', 6509),
   ('offers', 6229),
   ('black', 6165),
   ('to', 5492),
   ('in', 5309),
   ('that', 5244),
   ('on', 5209),
   ('it', 5015),
   ('cherry,', 4922),
   ('white', 4588),
   ('alongside', 4494),
   ('fruit', 4395),
   ('opens', 3887),
   ('tannins', 3758),
   ('tannins.', 3591),
   ('ripe', 3552),
   ('red', 3407),
   ('drink', 3377),
   ('but', 3143),
   ('by', 3118),
   ('dried', 3036),
   ('bright', 3022),
   ('delivers', 3009),
   ('finish.', 2947),
   ('cherry', 2926),
   ('spice', 2925),
   ('fresh', 2652),
   ('acidity', 2640),
   ("it's", 2583),
   ('an', 2446),
   ('notes', 2390),
   ('from', 2381),
   ('has', 2347),
   ('note', 2318),
   ('hint', 2315),
   ('shows', 2229),
   ('fruit,', 2203),
   ('berry,', 2114),
   ('acidity.', 2071),
   ('blend', 2012),
   ('berry', 2005),
 

(7) 이제 나라별로 출현빈도가 top-100에 드는 단어들만 남겨서 rdd6에 저장하도록 하자. rdd6의 키는 country가 되고 값은 단어들의 리스트가 되도록 하자. rdd5에 갖고 있던 단어의 출현빈도는 이제 필요없다. (10점)

In [29]:
rdd6 = rdd5.map(lambda x: ( x[0], x[1][:100]))
rdd6.take(1)

[('italy',
  [('and', 52395),
   ('the', 32396),
   ('of', 30661),
   ('a', 21991),
   ('with', 15931),
   ('this', 14488),
   ('aromas', 11805),
   ('palate', 9330),
   ('is', 8604),
   ('wine', 6509),
   ('offers', 6229),
   ('black', 6165),
   ('to', 5492),
   ('in', 5309),
   ('that', 5244),
   ('on', 5209),
   ('it', 5015),
   ('cherry,', 4922),
   ('white', 4588),
   ('alongside', 4494),
   ('fruit', 4395),
   ('opens', 3887),
   ('tannins', 3758),
   ('tannins.', 3591),
   ('ripe', 3552),
   ('red', 3407),
   ('drink', 3377),
   ('but', 3143),
   ('by', 3118),
   ('dried', 3036),
   ('bright', 3022),
   ('delivers', 3009),
   ('finish.', 2947),
   ('cherry', 2926),
   ('spice', 2925),
   ('fresh', 2652),
   ('acidity', 2640),
   ("it's", 2583),
   ('an', 2446),
   ('notes', 2390),
   ('from', 2381),
   ('has', 2347),
   ('note', 2318),
   ('hint', 2315),
   ('shows', 2229),
   ('fruit,', 2203),
   ('berry,', 2114),
   ('acidity.', 2071),
   ('blend', 2012),
   ('berry', 2005),
 

(8) 이제 오직 한 나라의 와인 설명에서만 등장하는 단어를 찾기 위해 나라마다의 출현빈도 top-100 단어에 등장하는 단어들이 몇 개 나라의 출현빈도 top-100에 속하는지를 세어보려고 한다. 이를 위해 우선 rdd6에 있는 각 나라별 단어리스트를 flatMap을 이용해 (word, country)의 쌍의 RDD로 변환시킨다. 그 결과는 rdd7이라 이름짓는다. (10점)

In [45]:
rdd7 = rdd6.flatMap(lambda x: (w[0], x) for w in x[1] )
rdd7.take(1)

NameError: name 'x' is not defined

(9) rdd7의 각 단어별로 groupByKey한다. 즉, 단어가 key가 되고 단어가 나오는 나라이름의 리스트가 value가 되는 RDD를 생성하여 rdd8이라 명명하시오. (10점)

In [ ]:
rdd 8 = rdd7.groupByKey().map(lambda x: (x[0], [1]))

(10) 마지막으로 오직 하나의 나라에서만 사용되는 단어의 수를 count하여 출력한다. (10점)